<a href="https://colab.research.google.com/github/PabloParadaSouto/Automatica/blob/master/Parte2_inverted_cart_pole_con_DQN_robotica_adaptativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero hacemos las instalaciones necesarias ....

In [1]:
!apt update
#!apt install xvfb
#!pip install pyvirtualdisplay
!pip install gym[toy_text]
!pip install pyglet~=1.3.2
!apt install -y graphviz
!pip install flax
!pip install graphviz
!pip install pyvirtualdisplay
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install 'gym[atari]'
!pip install -U dopamine-rl

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,353 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,848 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

Enlace importante:

https://colab.research.google.com/github/psc-g/intro_to_rl/blob/master/Introduction_to_reinforcement_learning.ipynb#scrollTo=inwNVOucQ6gr



Vamos a crear nuestra versión de la red DQN para el problema, para ello nos fijamos en la solución que aparece más abajo, y la reescribimos en nuestro cuaderno tratando de entender las cosas:


https://medium.com/analytics-vidhya/solving-open-ais-cartpole-using-reinforcement-learning-part-2-73848cbda4f1

https://github.com/maciejbalawejder/Reinforcement-Learning-Collection/blob/main/DQN/DQN.ipynb

Os pongo un ejemplo de partida:


In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import time
import random
from collections import deque

import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm
from tensorflow.keras import Sequential
from tensorflow.keras.activations import relu, linear
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
import math
from tensorflow.keras.regularizers import l2



class Model:
    def __init__(self, input_dim, output_dim, lr):
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.lr = lr
        #creamos dos redes, la principal y la target...
        self.Qpolicy = self.create()
        self.Qtarget = self.create()
        #Ahora copiamos la red principal en la target ...
        self.Qtarget.set_weights(self.Qpolicy.get_weights())

    def create(self):
        modelo = Sequential()
        modelo.add(Dense(512, input_dim = self.input_dim, activation = 'relu'))
        modelo.add(Dense(256, activation = 'relu'))
        modelo.add(Dense(128, activation = 'relu'))
        modelo.add(Dense(self.output_dim, activation = 'linear'))
        modelo.summary()
        modelo.compile(optimizer = RMSprop(learning_rate = self.lr, rho = 0.95, epsilon = 0.01), loss = "mse", metrics = ['accuracy'])
        #modelo.compile(optimizer='adam',loss='mean_squared_error', metrics=['mean_squared_error'])

        return modelo

class DQNAgent:
    def __init__(self, lr = 2.5e-4, gamma = 0.99, epsilon = 1, decay_coe = 0.99975, min_eps = 0.001,
                 batch_size = 64, memory_size = 10_000, episodes = 5_000, C = 5):


        self.env = gym.make('CartPole-v1')
        print ("entorno inicializado ")
        input ("pulsa para seguir ....")

        self.dim_estado = len(self.env.observation_space.low)
        print ("longitud del estado ",self.dim_estado)
        self.n_actions = self.env.action_space.n

        self.actions = [i for i in range(self.n_actions)]
        print ("acciones: ",self.actions)

        self.lr = lr
        self.gamma = gamma
        self.epsilon = epsilon
        self.decay_coe = decay_coe
        self.min_eps = min_eps
        self.episodes = episodes
        self.batch_size = batch_size
        self.D = deque(maxlen = memory_size) # replay memory
        self.C = C
        self.model = Model(self.dim_estado, self.n_actions, self.lr)


    def choose_action(self, state):

        #esto es para comprobar que entiendo la parte del codigo correspondiente al else
        debug_else=True


        if np.random.random() > (1 - self.epsilon) and not(debug_else):
            accion = np.random.choice(self.actions)
        else:
            estado = np.array(state)
            estado=estado.reshape(1,self.dim_estado)
            print ("comprobamos la forma del patron que le vamos a meter a la red ", estado.shape)
            input ("pulsa para seguir ...")
            accion = np.argmax(self.model.Qpolicy.predict(estado))
            print ("accion predicha ",accion)
            input ("hecho ")

        return accion


    def training(self):
        timestep_reward = 0

        for episode in range(self.episodes):
            print ("episodio: ",episode)
            #cogemos el estado
            estado_actual = self.env.reset()
            done = False
            score = 0
            while done != True:
                self.epsilon=self.epsilon*self.decay_coe
                accion = self.choose_action(estado_actual)
                estado_siguiente,refuerzo,is_done, info = self.env.step(accion)

                self.D.append([estado_actual, accion, refuerzo, estado_siguiente, is_done])
                #s_, r, done, _= self.env.step(a)

                # Rellenamos la pila de experimentos
                #self.es_estado_terminal = done
                #self.update_D(estado_actual,accion,refuerzo,estado_siguiente,done)

                #Es buena estrategia cambair el nombre de las variables
                print ("he ejecutado la accion ")
                estado_actual = estado_siguiente

                if len(self.D)>=self.batch_size:
                  aleatorio=random.sample(self.D, self.batch_size)

                  estados=self.state_shape([row[0] for row in aleatorio])
                  qvalue=self.model.Qpolicy.predict(estados)[0]

                  estados_=self.state_shape([row[3] for row in aleatorio])
                  qvalue_max=self.model.Qtarget.predict(estados_)[0]

                  for (estado_actual, accion, refuerzo, estado_siguiente, done) in aleatorio:

                    if done:
                      self.model.Qtarget=refuerzo
                    else:
                      self.model.Qtarget=refuerzo+self.gamma*np.max(qvalue_max)

              loss=(self.model.Qpolicy-self.model.Qtarget)**2/self.n_actions

                #self.train()



dqn = DQNAgent(episodes = 1_000)
dqn.training()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


entorno inicializado 
pulsa para seguir ....
longitud del estado  4
acciones:  [0, 1]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2560      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 2)                 258       
                                                                 
Total params: 167042 (652.51 KB)
Trainable params: 167042 (652.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "sequential_1"
________________________________________

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 24ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 25ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 24ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 38ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 23ms/step
accion predicha  0
hecho 
he ejecutado la accion 
compro

/usr/local/lib/python3.10/dist-packages/gym/envs/classic_control/cartpole.py:179: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 25ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 40ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 36ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 21ms/step
accion predicha  0
hecho 
he ejecutado la accion 
comprobamos la forma del patron que le vamos a meter a la red  (1, 4)
pulsa para seguir ...
1/1 [==============================] - 0s 24ms/step
accion predicha  0
hecho 
he ejecutado la accion 
compro

KeyboardInterrupt: Interrupted by user